In [17]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_k


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint


# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"  


In [ ]:

# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

# --- 프롬프트 템플릿 ---
template = """
당신은 영화 리뷰 감정 분석기입니다.
리뷰를 보고 감정을 반드시 "긍정", "부정", "보통" 중 하나로 분류하세요.

리뷰: {review}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# --- LLM 모델 불러오기 ---
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

print("모델 및 파서 설정 완료")


In [ ]:


# --- 테스트 리뷰 ---
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

print(f"테스트 텍스트 {len(test_reviews)}개 준비 완료")


In [ ]:

# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(test_review, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"review": test_review})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(test_reviews)
    
    for i, test_review in enumerate(test_reviews, 1):
        print(f"\n{i}. 텍스트: {test_review}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(test_review, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(test_review, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")
# 추가 기능: 배치 처리 및 통계
def batch_sentiment_analysis(text_list):
    """여러 텍스트를 한 번에 처리하고 통계 제공"""
    results = {
        '긍정': 0,
        '부정': 0,
        '중립': 0,
        '오류': 0
    }
    
    detailed_results = []
    
    for text in text_list:
        result, error = safe_sentiment_analysis(text)
        
        if result:
            sentiment = result.value
            results[sentiment] += 1
            detailed_results.append((text, sentiment, None))
        else:
            results['오류'] += 1
            detailed_results.append((text, None, error))
    
    return results, detailed_results

# 통계 출력 함수
def print_statistics(results, detailed_results):
    """결과 통계 출력"""
    print("\n=== 감정 분석 통계 ===")
    total = sum(results.values())
    
    for sentiment, count in results.items():
        percentage = (count / total * 100) if total > 0 else 0
        print(f"{sentiment}: {count}개 ({percentage:.1f}%)")
    
    print("\n=== 상세 결과 ===")
    for i, (text, sentiment, error) in enumerate(detailed_results, 1):
        status = sentiment if sentiment else " 오류"
        print(f"{i}. {text[:30]}... → {status}")

# 배치 처리 시뮬레이션
print("\n=== 배치 처리 시뮬레이션 ===")
simulated_results = {
    '긍정': 2,
    '부정': 2, 
    '중립': 1,
    '오류': 1
}

simulated_detailed = [
    (test_reviews[0], '긍정', None),
    (test_reviews[1], '부정', None),
    (test_reviews[2], '중립', None)
]

print_statistics(simulated_results, simulated_detailed)

    